In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import sys
sys.path.append('../src')
from torch import nn
from omegaconf import OmegaConf
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from tqdm import tqdm
pd.options.display.max_columns=100
from multiprocessing import Pool,cpu_count
from sklearn.metrics import pairwise_distances

In [2]:
data = {'num':torch.rand((8,40,52)),'cat':torch.cat([torch.ones((8,40,1),dtype=torch.long)]*3,dim=-1)}

In [ ]:
# nn.Conv1d()

In [ ]:
# mod(data['num']).shape

In [30]:
config = OmegaConf.load('../configs/lstm-transformer-v2.yaml')

In [31]:
import modellib

In [32]:
mod = getattr(modellib,config.model['class'])(config.model['kwargs'])

{'init_type': 'yakama'}
LSTMDpReLu(
  (rnn): LSTM(160, 512, batch_first=True, dropout=0.4, bidirectional=True)
  (dp): Dropout(p=0.4, inplace=False)
)
LSTMDpReLu(
  (rnn): LSTM(1184, 512, batch_first=True, dropout=0.4, bidirectional=True)
  (dp): Dropout(p=0.4, inplace=False)
)
LSTMDpReLu(
  (rnn): LSTM(2208, 512, batch_first=True, dropout=0.4, bidirectional=True)
  (dp): Dropout(p=0.4, inplace=False)
)
LSTMDpReLu(
  (rnn): LSTM(3232, 512, batch_first=True, dropout=0.4, bidirectional=True)
  (dp): Dropout(p=0.4, inplace=False)
)


In [34]:
# mod(data)

In [38]:
4256-96

4160

In [25]:
# mod(data)

In [11]:
# mod(data)

In [9]:
# mod

In [42]:
class CustomConformer(nn.Module):
    def __init__(self,in_dim,d_model,num_heads,dim_ff,dropout,kernel_size,num_kernels):
        super().__init__()
        self.proj_layer = nn.Linear(in_features=in_dim,out_features=d_model,bias=False)
        self.reverse_proj_layer = nn.Linear(in_features=d_model,out_features=in_dim,bias=False)        
        self.encoder = nn.TransformerEncoderLayer(d_model=d_model,nhead = num_heads,dim_feedforward=dim_ff,dropout=dropout)
        self.cnn = nn.Conv1d(in_channels=in_dim,out_channels=num_kernels,kernel_size=kernel_size,padding=(kernel_size//2))
    def forward(self,x):
        x = self.proj_layer(x)
        x = self.encoder(x)
        x = self.reverse_proj_layer(x)
        x_cnn = self.cnn(x.permute(0,2,1)).permute(0,2,1)
        x = torch.cat([x,x_cnn],dim=-1)
        return x

In [ ]:
nn.MultiheadAttention()

In [43]:
mod = CustomConformer(in_dim=40,d_model=1024,num_heads=128,dim_ff=1024,dropout=0.1,kernel_size=3,num_kernels=16)

In [44]:
mod(data['num']).shape

torch.Size([8, 40, 40])


torch.Size([8, 40, 56])

In [8]:
m = nn.TransformerEncoderLayer(d_model=32,nhead=1,dim_feedforward=128,)

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = inputs
    x = tfa.layers.MultiHeadAttention(
        head_size=head_size,
        num_heads=num_heads,
        use_projection_bias = False,
        dropout=Config.DROPOUT
    )([x, x, x])

    res = WeightedSum()([x, inputs])
    res = tf.keras.layers.LayerNormalization(epsilon=1e-6)(res)

    # Feed Forward Part
    x = tf.keras.layers.Dense(ff_dim, activation='relu')(x)
    x = tf.keras.layers.Dropout(Config.DROPOUT)(x)

    x = tf.keras.layers.Dense(inputs.shape[-1])(x)
    x = tf.keras.layers.Dropout(Config.DROPOUT)(x)
    x = WeightedSum()([x, res])

    return tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

In [ ]:
class CustomTransformerEncoderLayer(nn.Module):
    def __init__(self,d_model,nhead,dim_feedforward,dropout,activation='relu'):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim=d_model,num_heads=nhead,dropout=dropout)        
        self.linear1 = nn.Linear(in_features=d_model,out_features=dim_feedforward)
        self.linear2 = nn.Linear(in_features=dim_feedforward,out_features=d_model)        
        self.act = nn.ReLU(inplace=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)        
    def forward(self,x):
        attn = self.self_attn(x,x,x)
        res = x + attn
        res = self.norm1(res)
        x = self.linear1(attn)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.dropout(x)
        res = x+res
        x = self.norm2(x)
        return x

In [ ]:
nhead,
    dim_feedforward=2048,
    dropout=0.1,
    activation='relu',

In [ ]:
nn.TransformerEncoderLayer()